## Import of required Modules
Make sure, that the imported moduls are installed.

In [96]:
import pickle
import re
from tqdm import tqdm
import time
import numpy as np
from IPython.display import clear_output 
import pandas as pd
import time
from selenium import webdriver
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from selenium.common.exceptions import NoSuchElementException, TimeoutException

### Settings
change the settings to assert the scraper runs on your system. We reccomend to change the wait_time to a higher value if you run the scraper on a slower system or internet connection.

In [97]:
wait_time = 0.25

#### set up Selenium and Chrome Driver 
We use selenium with Chrome and tested the scraper with the chromedriver. You need the latest version of the driver from https://chromedriver.chromium.org/. Alternatively, change to the driver to a driver of your preferance.
We set up the scraper to run in the background, if you wish to run it in regular window mode, remove the line  "chrome_options.add_argument("--headless")".

In [98]:
# chromedriver setup

serv = Service(r"D:/Wenjie_workspace/causal_search engine/Qbias_dataset/Qbias-main/Scraper/driver/chromedriver.exe") #path from 'which chromedriver'
#D:\Wenjie_workspace\causal_search engine\Qbias_dataset\Qbias-main\Scraper\driver
# test driver
# for headless chrome mode
chrome_options = Options()

# remove this line if you do not wish to run in background 
chrome_options.add_argument("--headless") 

In [99]:
# assemble list of links to all articles
links = []

# interact with cookie terms
chrome_path = r"C:/Program Files/Google/Chrome/Application/chromedriver.exe" #path from 'which chromedriver'
#C:\Program Files\Google\Chrome\Application
#D:\Wenjie_workspace\causal_search engine\Qbias_dataset\Qbias-main\Scraper\driver
chrome_options = Options()
chrome_options.add_argument("--headless") # open chrome in background
driver = webdriver.Chrome(service=serv, options=chrome_options)
#driver=webdriver.Chrome()                #实例化1个谷歌浏览器对象
#driver.get('https://www.baidu.com/')     #开始

### Retrieve Article Links for Crawl

In [55]:
# assemble list of links to all articles
links = []

# interact with cookie terms
chrome_path = r"C:/Program Files/Google/Chrome/Application/chromedriver.exe" #path from 'which chromedriver'
#C:\Program Files\Google\Chrome\Application
#D:\Wenjie_workspace\causal_search engine\Qbias_dataset\Qbias-main\Scraper\driver
chrome_options = Options()
chrome_options.add_argument("--headless") # open chrome in background
driver = webdriver.Chrome(service=serv, options=chrome_options)
driver.get('https://www.allsides.com/headline-roundups')
#wait = WebDriverWait(driver, 10)
#wait.until(EC.element_to_be_clickable((By.CLASS_NAME, 'css-47sehv')))
#ele = driver.find_element(By.XPATH, "//button[contains(.,'')]").click()
#ele = driver.find_element(By.CLASS_NAME, "css-47sehv")
#ele.click()
#driver.find_element(By.XPATH, "//button[contains(.,'AGREE')]").click()
# retireve number of pages 
last_page_button = driver.find_element(By.CLASS_NAME, "pager-last")
link_last_page = last_page_button.find_elements(By.TAG_NAME, "a")
t = link_last_page[0].get_attribute("href")
last_page_index = int(t[-3:])

# retrieve links from start page
main_table = driver.find_element(By.XPATH, "//*[@id=\"block-views-de37fa32ea86f5545eb9b7722977a70d\"]/div/div[2]/table/tbody") # table body

rows = main_table.find_elements(By.TAG_NAME, "tr")
for i in rows:
    entry = i.find_elements(By.TAG_NAME, "td")
    link = entry[0].find_element(By.TAG_NAME, "a")
    links.append(link.get_attribute("href"))
WebDriverWait(driver, 20)

# retrieve links for other pages
for page in tqdm(range(2,last_page_index+1)):   # set to max number of pages
    driver.get("https://www.allsides.com/headline-roundups?page="+str(page))
    WebDriverWait(driver, 20)
    main_table = driver.find_element(By.XPATH, "//*[@id=\"block-views-de37fa32ea86f5545eb9b7722977a70d\"]/div/div[2]/table/tbody") # table body

    rows = main_table.find_elements(By.TAG_NAME, "tr")
    for i in rows:
        entry = i.find_elements(By.TAG_NAME, "td")
        link = entry[0].find_element(By.TAG_NAME, "a")
        links.append(link.get_attribute("href"))

driver.close()

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 184/184 [08:24<00:00,  2.74s/it]


In [56]:
# exports links as pickle file
with open("linklist_allsides_news.pickle", "wb") as f:
    pickle.dump(links, f, pickle.HIGHEST_PROTOCOL)

In [57]:
# exports links as csv
with open("linklist_allsides_news.csv", "w") as f:
    for line in links:
        print(line, file=f)

In [58]:
# functions that assert the existence of 
def check_exists_by_xpath(xpath):
    try:
        driver.find_elements(By.XPATH, xpath)[0]
    except:
        return False
    return True

def check_exists_by_class(inp):
    try:
        driver.find_elements(By.CLASS_NAME, inp)[0]
    except:
        return False
    return True

## Retrieve Articles
This section of the crawler retrieves all available news articles from AllSides along with the available information and bias tags.

In [125]:
# load link list from pickle file
with open("linklist_allsides_news(0.52).pickle", "rb") as f:
    links = pickle.load(f)

In [126]:
with open("linklist_allsides_news(0.52).pickle", "rb") as file:
    data_1 = pickle.load(file)

In [127]:
updated_data = data_1[902:]

# Save the updated list back to a pickle file
updated_file_path = "linklist_allsides_news(last_2566).pickle"
with open(updated_file_path, "wb") as file:
    pickle.dump(updated_data, file)

In [128]:
with open("linklist_allsides_news(last_2566).pickle", "rb") as f:
    links = pickle.load(f)

In [129]:
# List for storing results
data = []

# Retrieve information from each article URL
for li in tqdm(links):
    driver.get(li)
    time.sleep(2)  # Adjust sleep time as necessary for page loads

# 检查并关闭弹窗
    try:
        popup_button = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CLASS_NAME, "your-popup-class-name"))
        )
        popup_button.click()
    except TimeoutException:
        print("弹窗未出现或无需点击")


    # Collecting headline data
    try:
        heading = driver.find_element(By.TAG_NAME, "h1").text
    except NoSuchElementException:
        heading = "No heading found"

    # Collecting date
    try:
        date = driver.find_element(By.CLASS_NAME, "date-display-single").text
    except NoSuchElementException:
        date = ""

    # Collecting tags
    try:
        tags_elements = driver.find_elements(By.CLASS_NAME, "page-tags")
        tags = [a.text for elem in tags_elements for a in elem.find_elements(By.TAG_NAME, "a")]
    except NoSuchElementException:
        tags = []

    # XPath for different article sections
    divs = [
        "/html/body/div[3]/div/div/div/div[4]/div/div/div/div[1]",
        "/html/body/div[3]/div/div/div/div[4]/div/div/div/div[2]",
        "/html/body/div[3]/div/div/div/div[4]/div/div/div/div[3]"
    ]

    # Process each article section
    for d in divs:
        if check_exists_by_xpath(d):
            div = driver.find_element(By.XPATH, d)
            try:
                cat = div.find_element(By.TAG_NAME, "h3").text
            except NoSuchElementException:
                print("No headline found in div")
                continue

            try:
                link = div.find_element(By.TAG_NAME, "a").get_attribute("href")
            except NoSuchElementException:
                print("No link found in div")
                continue

            # Collect and store data based on bias category
            bias_category = "center" if "Center" in cat else "left" if "Left" in cat else "right"
            try:
                news_heading = div.find_element(By.CLASS_NAME, "news-title").text
            except NoSuchElementException:
                news_heading = ""

            try:
                source = div.find_element(By.CLASS_NAME, "source-area").find_element(By.TAG_NAME, "span").text
            except NoSuchElementException:
                source = ""

            try:
                news_text = div.find_element(By.CLASS_NAME, "news-body").find_element(By.CLASS_NAME, "body-contents").text
            except NoSuchElementException:
                news_text = ""

            # Append article data
            data.append({"url": link, "date": date, "main_title": heading, "tags": tags, "section_heading": news_heading, "source": source, "text": news_text, "bias_rating": bias_category})
        else:
            print("Article section not found")



  0%|                                                                                                                                                                                                         | 0/2566 [00:00<?, ?it/s]

弹窗未出现或无需点击


  0%|                                                                                                                                                                                              | 1/2566 [00:17<12:37:45, 17.73s/it]

弹窗未出现或无需点击


  0%|▏                                                                                                                                                                                             | 2/2566 [00:35<12:41:38, 17.82s/it]

...

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▉| 2565/2566 [11:36:54<00:16, 16.57s/it]

弹窗未出现或无需点击
No headline found in div
No headline found in div
No headline found in div


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2566/2566 [11:37:09<00:00, 16.35s/it]

弹窗未出现或无需点击


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2566/2566 [11:37:09<00:00, 16.30s/it]


In [130]:
# Close the driver after all processing
# 将数据转换为 pandas DataFrame
df = pd.DataFrame(data)

# 保存 DataFrame 到 CSV 文件
# df.to_csv("allsides_news_complete.csv", index=False)
# driver.quit()

In [104]:
print(data)

[{'url': 'https://www.cnn.com/2022/11/10/europe/kherson-gains-russia-retreat-intl/index.html', 'date': 'November 9th, 2022', 'main_title': 'Russia Begins Retreat From Kherson, Ukraine Warns of Traps Left Behind', 'tags': ['World', 'Ukraine War', 'Russia', 'Ukraine', 'Volodymyr Zelenskyy', 'Vladimir Putin', 'Europe'], 'section_heading': 'Ukraine claims big gains in south, but fears retreating Russians will turn Kherson into ‘city of death’', 'source': 'CNN (Online News)', 'text': 'Ukraine’s military said it had retaken swathes of territory in Kherson on Thursday after Moscow ordered a partial withdrawal from the area, though officials in Kyiv warned that retreating Russian soldiers could turn the regional capital into a “city of death” on their way out.\nA military spokesman said that in just 24 hours, Ukrainian forces had pushed the front line in the key southern region of Kherson forward by 7 kilometers (4.3 miles) and taken control of more than 260 square kilometers of territory (100

In [131]:
## convert data to dataframe
#df = pd.DataFrame(data)

# export scraped articles
df.to_csv("allsides_news_complete(last_2566).csv")